In [1]:
%config IPCompleter.greedy=True
import numpy as np
import cv2
import ModelCheck as mc
from Card import Card

Instructions for updating:
Colocations handled automatically by placer.


In [2]:
def find_cards(og_frame, pp_frame):
    
    #List holding valid cards
    found_cards = []
    
    #Find all contours from given frame
    contours, hierarchy = cv2.findContours(pp_frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Make black image canvas for plotting contours
    contour_image = np.zeros(pp_frame.shape, dtype=np.uint8)
    
    
    
    #Go through each contour as c
    for c in contours:
        
        size = cv2.contourArea(c)                       # areal
        peri = cv2.arcLength(c, True)                   # omkreds
        approx = cv2.approxPolyDP(c, 0.01*peri, True)   # reducere antal hjørner -> prøv at sæt den op til 0.05 (var 0.01)
    
        is_rectangle = len(approx) == 4                 # har 4 hjørner/punkter(vertices)
        is_reasonable_size = size>4000 and size<100000  # er firkanten en realistisk størrelse
        #is_parent_contour = hierarchy[0,idx,3] == -1    # hvis det giver -1 er det den øverste figur i hierarkiet
        
        
        if(is_rectangle and is_reasonable_size):
            #cv2.drawContours(contour_image, contours , idx ,(0,255,0), 0)
            cv2.fillPoly(contour_image, pts=[approx], color=(255))
            
    
    
    
    for c in cv2.findContours(contour_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]:
        
        peri = cv2.arcLength(c, True)                   # omkreds
        approx = cv2.approxPolyDP(c, 0.01*peri, True)   # reducere antal hjørner -> prøv at sæt den op til 0.05 (var 0.01)
        
        if(len(approx) == 4 ):        # Prøv at lav figuren til et kort
            
            card = Card()
        
            card.contour = c
            
            card.size = size
            
            card.coords = list(approx.reshape(4,2))
            
            card.width, card.height = cv2.boundingRect(c)[2:]
            
            card.set_center()
            
            card.find_warped_card(og_frame) # Find og extract kortet fra billedet og sæt det i self.warped 

            card.find_icons()
        
            if card.validate(): found_cards.append(card) # Hvis kortet er validt. Tilføj til found_cards
        
    
    found_cards.sort(key=lambda x: [x.num, x.suit])
    
#     cv2.imshow("og_image", og_frame)
#     cv2.imshow("pp_image", pp_frame)
    cv2.imshow("contour_image", contour_image)
#     cv2.imshow("warped", found_cards[2].warped)
#     cv2.imshow("warped_thresh", found_cards[2].warped_thresh)
#     cv2.imshow("suit_img", found_cards[2].suit_img)
#     cv2.imshow("num_img", found_cards[2].num_img)
    
    
#     cv2.imshow("abs.diff h", cv2.absdiff(found_cards[2].suit_img, cv2.imread("./test_cards/h.jpg", cv2.IMREAD_GRAYSCALE)))
#     cv2.imshow("abs.diff c", cv2.absdiff(found_cards[2].suit_img, cv2.imread("./test_cards/c.jpg", cv2.IMREAD_GRAYSCALE)))
#     cv2.imshow("abs.diff d", cv2.absdiff(found_cards[2].suit_img, cv2.imread("./test_cards/d.jpg", cv2.IMREAD_GRAYSCALE)))
#     cv2.imshow("abs.diff s", cv2.absdiff(found_cards[2].suit_img, cv2.imread("./test_cards/s.jpg", cv2.IMREAD_GRAYSCALE)))
    
    
        
    return found_cards


In [3]:
def draw_predicted_hand(img, text):

    FONT = cv2.FONT_HERSHEY_SIMPLEX
    TEXT = text
    FONT_SIZE = 2
    FONT_COLOR = (255, 0, 255)
    THICKNESS = 5

    ######

    # get boundary of this text
    text_w, text_h = cv2.getTextSize(TEXT, FONT, FONT_SIZE, THICKNESS)[0]
    
    img_h, img_w, _ = img.shape
    
    text_x = int(img_w/2 - text_w/2)
    text_y = int(img_h - text_h)

    cv2.putText(img, TEXT, (text_x, text_y), FONT, FONT_SIZE, FONT_COLOR, thickness=THICKNESS)

    
def draw_warps_and_icons_from_cards(cards):
    
    # [Height,Width,Depth]
    # [1080,  1920, 3]
    
    # Hvis der ikke en nogle kort lavet en bare et sort canvas
    if(len(cards)) == 0: return np.zeros([540,1920,3], dtype=np.uint8)
    
    cards = cards[:5]
    
    ########################################
    
    # Lav et img med alle kort
    res1 = cards[0].warped_thresh
    
    for card in cards[1:]: 
        res1 = np.hstack((res1, card.warped_thresh))
    
    # Black canvas til at fylde overskydende plads ud
    black = np.zeros((res1.shape[0], cards[0].warped_thresh.shape[1]*5 - res1.shape[1]), dtype=np.uint8)
    
    res1 = np.hstack((black, res1))               # Stack vores kort og vores sorte canvas
    res1 = cv2.cvtColor(res1, cv2.COLOR_GRAY2RGB) # Convert til farve RGB
    res1 = cv2.resize(res1, (960, 540))           # Resize til 1/4 af skærmstørrelse
    
    ########################################

    res2 = cards[0].warped
    for card in cards[1:]: res2 = np.hstack((res2, card.warped))
        
    black = np.zeros((res2.shape[0], cards[0].warped.shape[1]*5 - res2.shape[1], 3), dtype=np.uint8)
    res2 = np.hstack((res2, black))
    res2 = cv2.resize(res2, (960, 540))
    
    ########################################
    
    return np.hstack((res1,res2))


In [4]:

def preprocess_frame(frame):
    
    threshold = 160
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    thresh = cv2.threshold(blur,threshold,255,cv2.THRESH_BINARY)[1]
    return thresh

def preprocess_frame_clahe(frame):

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8,8))
    cl1 = clahe.apply(gray)
    thresh = cv2.threshold(cl1, 160 ,255,cv2.THRESH_BINARY)[1]
    return thresh

def preprocess_frame_adap(frame):
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gaus = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 401, 1)
    return gaus



def find_cards_from_image(frame): # returns new frame & list of all cards found
    
    # prepross_frame = preprocess_frame(frame) 
    # prepross_frame = preprocess_frame_clahe(frame)
    prepross_frame = preprocess_frame_adap(frame)
    
    found_cards = find_cards(frame, prepross_frame) # returns a list of cards from the processed frame
    
    for card in found_cards:
        
        card.draw_card_outlines(frame) # outliner kortet
        card.draw_text(frame)          # tegner suit osv. over kortet


    predicted_hand = mc.getHand(found_cards)
    
    draw_predicted_hand(frame, predicted_hand)
    
    ###########################33
    
    bottom = draw_warps_and_icons_from_cards(found_cards) # dims = [540, 1920, 3]
    
    top = np.hstack((cv2.cvtColor(prepross_frame,cv2.COLOR_GRAY2RGB), frame))
    top = cv2.resize(top, (1920, 540))
    
    res = np.vstack((top, bottom))
    res = cv2.resize(res, (0,0), fx=0.8, fy=0.8)
    
    return found_cards, res  

In [ ]:
def save_img_icon(img_path, save_path, suit=True, num=True):
    
    found_cards = find_cards(cv2.imread(img_path), preprocess_frame(img)) # returns a list of found cards
    
    if(len(found_cards) == 0): return
    
    card = found_cards[0] # take first card (there should only be one)
    
    if suit: cv2.imwrite(save_path, card.suit_img)
    if num: cv2.imwrite(save_path, card.num_img)

print("yes2")

yes2


In [ ]:
def use_img(img):
    img = cv2.imread(img)
    
    cards, preview = find_cards_from_image(img)
    
#     cv2.namedWindow("Feed", cv2.WINDOW_NORMAL)
#     cv2.setWindowProperty("Feed", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
#     cv2.namedWindow("Feed2", cv2.WINDOW_NORMAL)
#     cv2.setWindowProperty("Feed2", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    print(img.shape)
    
#     cards[0].draw_warped
#     cards[0].draw_suit
#     cards[0].draw_num
    
    cv2.imshow("Feed", img)
    cv2.imshow("Feed2", preview)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
def use_cam(num=0):
    cap = cv2.VideoCapture(num)

    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'): break
        ret, frame = cap.read()
        cards, preview = find_cards_from_image(frame)
        
        cv2.imshow("Feed", frame)
        cv2.imshow("Feed2", preview)

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(0)

In [ ]:
#use_img("imgs/img8.jpg")
use_cam(1)
#save_img_icon("./pre/clubs.jpg", "./delete/this2.jpg", suit=False, num=True)

1.0303567091531387 1.0192598405519675 13
1.0302453132169083 1.0130217854394858 13
1.0155547013092654 1.012765031719625 13
1.0155547013092654 1.012765031719625 13
1.000864491192354 1.012469019876399 13
1.0151320503189598 1.0128782121447777 13
1.0001081256423119 1.006230348011244 7
1.0001081256423119 1.006230348011244 7
1.0001081256423119 1.006230348011244 11
1.0001081256423119 1.006230348011244 11
1.0004324324526441 1.0125386920751691 4
1.014583182421713 1.005936170075765 7
1.0004324324526441 1.0125386920751691 7
1.014583182421713 1.005936170075765 10
1.000324271748241 1.0062692842413203 10
1.0 1.0 10
1.014583182421713 1.005936170075765 10
1.0118633323304669 1.0185798617977138 10
1.0279651989027025 1.0120735112000898 10
1.0461111536330632 1.000866442837734 10
1.0460578989765992 1.0007512787243873 10
1.0001081256423119 1.006230348011244 10
1.0001081256423119 1.0062257561668457 10
1.047607325438469 1.014839779119271 7
1.16 1.101414887360054 10
1.0336965291773168 1.0093471103923612 10
1.29

1.0322969184814976 1.040353969968392 10
1.0191923734307502 1.0059714908964688 2
1.0108509967313177 1.0180434266528953 7
1.0282701247927437 1.0107490300427184 1
1.0191513325652077 1.0046026230794676 12
1.060430864638069 1.0161258432562599 10
1.0302392442216288 1.0063352972705035 13
1.0268297627741472 1.0220724469682316 2
1.01924117757686 1.0175994125877896 7
1.0036988854455893 1.0315997420376701 1
1.0376140679123136 1.0029149666725115 12
1.0187717349801035 1.0153686282958685 13
1.0259932500169524 1.0136722342881177 2
1.002246672945796 1.0185315330369127 7
1.001708075778935 1.012530533311556 1
1.0269019774870722 1.0025716464093126 12
1.023989998428358 1.006885755405445 13
1.027168606477657 1.0301406708307932 2
1.02884639823497 1.0232595759482588 7
1.0199946593283544 1.0176248131462315 1
1.0200537423946876 1.0107874613619454 12
1.0246323473264385 1.0154156880580507 None
1.0140871823861695 1.0236401063786909 2
1.0043730202137344 1.0310778360724162 7
1.0095895783123354 1.0056375456320725 1


1.0184454422136726 1.0074842340486507 13
1.0055363982849461 1.0088198496756453 2
1.007801895332818 1.0064933351479788 7
1.0006178560984906 1.00602881416744 1
1.0156304696468819 1.0071833183350218 12
1.0238978960740193 1.001091926893238 13
1.0169098074673357 1.0181805473279844 2
1.0158089424753383 1.0127906326039502 7
1.007801895332818 1.00644134837835 1
1.0229284288586757 1.0138016364605222 12
1.0306842001057233 1.0149662896345977 13
1.0170739106865425 1.0102489178940974 2
1.0169943041480476 1.0008564700734908 7
1.0163871539764446 1.0068136311992317 1
1.0073857257336105 1.0065022009223545 12
1.0181965306161858 1.0076885637987256 13
1.0062517784011316 1.0076463541898404 2
1.0161628775570593 1.0067477509175942 7
1.0005366054726423 1.005931427583622 1
1.0073857257336105 1.0064469326846348 12
1.0178064883339828 1.0009060556565987 10
1.0113152689440044 1.0095300865131367 2
1.000607479196266 1.005927046104748 7
1.0103554664490568 1.0175712572286553 1
1.0065618605943905 1.0125023858315434 12


1.0656802115542467 1.0318571063064488 1
1.043397757994822 1.0214800326567854 2
1.0389513731760742 1.0092172579875773 7
1.038862102232833 1.0210119168848524 1
1.0837176569004905 1.0119782172490448 12
1.052612299186481 1.031763800038514 1
1.0439201635775464 1.0367182122095746 2
1.049719404424572 1.0041840132320343 7
1.045328870985793 1.0161079385553904 12
1.048750451115863 1.0059192545763072 1
1.0589042170498633 1.0145866836371653 1
1.0565264757658932 1.019423053387921 2
1.0690187909098303 1.0109365817539409 7
1.0547912026254822 1.017110956580644 12
1.0688266785920437 1.0054724691886012 1
1.0456360252261783 1.0244093978741933 1
1.050306696453276 1.0392156862745099 2
1.0494860037759002 1.002666298964379 7
1.0391255096980545 1.0205916186144917 1
1.0833956728907392 1.0250358270863458 12
1.0454526705781686 1.002463684002924 1
1.0498340760961125 1.0395979091953078 2
1.0493459384679238 1.0097671911431834 7
1.048037867552733 1.0011308667306411 1
1.0743815348897212 1.010750036573663 12
1.0658706

1.1242685164933919 1.0007390265307883 1
1.1043853181780068 1.011591820283766 2
1.109493936165259 1.0492182631377784 7
1.1219195521036958 1.0055132683873247 1
1.1240575617649056 1.0098819691902126 12
1.0990242973338982 1.0082813501014938 2
1.1459422542044668 1.0572569707058108 7
1.1019891946670437 1.023210577974794 1
1.127529109063561 1.0092378243492346 12
1.1012122089665812 1.0095585852874573 2
1.121914031331397 1.0589789917643073 7
1.1243137828873562 1.0130114898543034 1
1.1173746625516938 1.0012774893428489 12
1.1270380476742914 1.0051527139294723 1
1.0871730024044506 1.011802064587389 2
1.131728922724036 1.0412877391201834 7
1.118033988749895 1.029969918902331 1
1.1345759080834468 1.0051908452190852 12
1.117719788843139 1.0442225555757543 1
1.1027910358130084 1.0211916737969473 2
1.1242725164511926 1.0516267126148755 7
1.1405529978970028 1.0008496178253068 1
1.1364066694380122 1.0033610784603375 12
1.117301640575935 1.0257715224586024 1
1.088834397338542 1.0068951473901528 2
1.13710

...

1.1471151797175094 1.0273502784017496 1
1.133950878386976 1.0470664535581868 2
1.1650340895369642 1.034375948469106 12
1.1381549410916951 1.0345773023571168 7
1.1481729747432112 1.0166638290242704 1
1.1286356720696487 1.0439538554602337 2
1.1649907423477492 1.0354582804108163 12
1.1695165701660222 1.0388690060144472 7
1.1488045930070456 1.00326073883699 1
1.1402878821134235 1.063910999466587 2
1.1650340895369642 1.0338167439724617 12
1.1597680695768082 1.036545091837275 7
1.1420698195738193 1.0288569320760013 1
1.1336442938090256 1.0546493904876684 2
1.1618911009762676 1.0338379200762018 12
1.148051195505866 1.0363434672448932 7
1.1353550730359696 1.002572087822606 1
1.1402728884856559 1.0571147745597858 2
1.1650340895369642 1.0338167439724617 12
1.148008538430573 1.0439198079437344 7
1.1479698974765058 1.0098680406371807 1
1.1469945323486737 1.0663585806697922 2
1.174757281553398 1.0341006536356656 12
1.1575084364970343 1.0549616876628973 7
1.136233349524634 1.0112091984341864 1
1.139

1.1400540088870394 1.013237001826456 1
1.1321797970767755 1.044874240112383 2
1.1357497292405945 1.0385300730757685 7
1.1440596393590463 1.0046632902544954 1
1.1525260331724687 1.0199874582447566 12
1.1402353933012042 1.0032335556400334 1
1.120748598508865 1.0410279799382562 2
1.1551628482893788 1.0409964508251186 7
1.1440596393590463 1.0046632902544954 1
1.1617818969158755 1.0302535677167557 12
1.1472802764807006 1.0034549302310452 1
1.1136763635696283 1.0494055468440118 2
1.1454562858111308 1.040002353984729 7
1.1457416137283487 1.0106747971080627 1
1.1520682239681728 1.0356537067985994 12
1.1530562736851853 1.0046917147662024 1
1.1256552791028425 1.0532827594061305 2
1.145672205942325 1.0225149048941031 7
1.1454562858111308 1.0056567183342466 1
1.1504724759338076 1.036148746249511 12
1.1338550904519773 1.006676087225213 1
1.1136763635696283 1.0494055468440118 2
1.1454562858111308 1.040002353984729 7
1.1454562858111308 1.0056567183342466 1
1.1712989679969281 1.0315293835052683 12
1.1

1.1580395448626513 1.0120493996649456 1
1.1389405307786225 1.034786179095794 2
1.1824736752276073 1.0391597520644866 7
1.188804537238148 1.0271334052306407 12
1.1508305041431341 1.0010536022940848 1
1.1340624652744344 1.0149922345035518 1
1.1334812115039623 1.0423904264912434 2
1.1774015197039793 1.0408880905336981 12
1.1505213696256658 1.0519653620528642 7
1.1618480088341268 1.0046047826039697 1
1.139035361770448 1.0432589619768269 2
1.1697552744635227 1.0315733246877907 12
1.188203466953157 1.0501633361377993 7
1.1499424372276936 1.0122321513219834 1
1.1444807190028385 1.0345352789629334 2
1.1667972566774063 1.0220108477112686 12
1.1573571136497145 1.0534981003412776 7
1.1706181652736993 1.025232186516513 1
1.1602164416881002 1.0257397839043823 1
1.1460630995095515 1.0283153370598108 2
1.1681697750829907 1.0315293835052683 12
1.1771445965945195 1.0562465150146119 7
1.1676330035735834 1.0004504842934407 1
1.1388888888888888 1.03112030150465 2
1.1668136297795084 1.0347118155843154 7
1.

1.0086993032107885 1.03299852559353 1
1.0455815424882922 1.0128096205863757 2
1.021955509591203 1.0165192088932256 12
1.034822204706509 1.0159217753971856 1
1.0232278697650272 1.000985638087758 7
1.0615491779008004 1.0769535556624388 2
1.0468302911821883 1.0013863222922452 None
1.0096865222104845 1.0403088547395303 4
1.0453628581285253 1.0013203068860774 4
1.0305264925561606 1.0550730736548275 2
1.0385347619724714 1.0228611188178374 1
1.0208122548709644 1.0326924392680552 13
1.0118376350989158 1.0161448398582114 1
1.011154468775924 1.0086679844856803 7
1.0226046801692201 1.022429455243845 2
1.0393103016817808 1.0020652035569158 1
1.0224104649047152 1.0322460007015857 12
1.0221788882799063 1.0005370209153943 1
1.0229615098945732 1.0009534189035691 13
1.0304850208420124 1.0339768334658774 2
1.0401530912995176 1.0314277274577732 1
1.0439876474075296 1.0249063095492308 12
1.034582439799878 1.0071223555627191 1
1.023973082484202 1.0071905008108908 7
1.030254587040047 1.0339768334658774 2
1.

1.034097129207678 1.0074842340486507 1
1.0314848229053302 1.0210308724539343 1
1.0298777846331617 1.0127043342603117 2
1.0344648484981456 1.0011067700803302 7
1.0427195936316707 1.0179727096145328 12
1.0339962596567875 1.0076301836449755 1
1.039097512619943 1.020233516563243 1
1.0221368977983347 1.0340425936106494 2
1.0344648484981456 1.0011067700803302 7
1.0429805042999687 1.0175802019148656 12
1.0118376350989158 1.0160156907203892 1
1.0391999513716912 1.0097804587064256 1
1.0296280273402219 1.0234668800181392 2
1.0322543935232134 1.0006506282348362 12
1.0222627463184608 1.0078631271011007 1
1.0340519865553215 1.0011067700803302 7
1.0312390177788313 1.0212541263359691 1
1.0221369848441764 1.0337345252080288 2
1.0319697287470446 1.008851702534897 12
1.023229233000418 1.015818204863524 7
1.044592150872562 1.0086544662940373 1
1.0232160725346493 1.0102272123433522 1
1.0374208040640183 1.0234811026071524 2
1.0315283704270866 1.017376642519258 12
1.0227024282548784 1.0159905290826414 1
1.0